# Convolutional Neural Network

Notebook inspired by https://github.com/aymericdamien/TensorFlow-Examples/
and by https://github.com/pkmital/CADL.

Example is using the [MNIST database of handwritten digits](http://yann.lecun.com/exdb/mnist/)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [ ]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

In [ ]:
mnist

In [ ]:
img = mnist.train.next_batch(1)
img[0].shape

In [ ]:
img = img[0].reshape((28, 28))

In [ ]:
plt.imshow(img, cmap='Greys')

## Convolution with a gaussian kernel

Let's define a gaussian kernel and let's convolve the image with it.

In [ ]:
ksize = 7

I'll start by defining a 1 dimensional gaussian

In [ ]:
x = tf.linspace(-1.0, 1.0, ksize)

In [ ]:
# The 1 dimensional gaussian takes two parameters,
# the mean value, and the standard deviation
mean = 0.0
std = 0.5
gaussian = (tf.exp(tf.negative(tf.pow(x - mean, 2.0) /
                          (2.0 * tf.pow(std, 2.0)))) *
            (1.0 / (std * tf.sqrt(2.0 * 3.1415))))

In [ ]:
with tf.Session() as sess:
    res = gaussian.eval()
plt.plot(res)

In [ ]:
# Let's store the number of values in our Gaussian curve.
ksize = gaussian.get_shape().as_list()[0]

# Let's multiply the two to get a 2d gaussian
gaussian_2d = tf.matmul(tf.reshape(gaussian, [ksize, 1]), tf.reshape(gaussian, [1, ksize]))

# Execute the graph
with tf.Session() as sess:
    gaussian_2d_res = gaussian_2d.eval()

plt.imshow(gaussian_2d_res, cmap='Greys')

In [ ]:
gaussian_2d_res

Ok, now that we have a 2d Gaussian, we'd like to convolve it with the image. Remember that in order to do that we need to reshape both the image and the kernel as 4D tensors.

In [ ]:
img_4d = tf.reshape(img, [1, img.shape[0], img.shape[1], 1])
print(img_4d)

In [ ]:
print(img_4d.get_shape())

In [ ]:
z_4d = tf.reshape(gaussian_2d, [ksize, ksize, 1, 1])
print(z_4d.get_shape())

The convolution operation is performed in tensorflow using the `nn.conv2d` function

In [ ]:
convolved = tf.nn.conv2d(img_4d, z_4d,
                         strides=[1, 1, 1, 1],
                         padding='SAME')
convolved

In [ ]:
with tf.Session() as sess:
    res = convolved.eval()
print(res.shape)

In [ ]:
plt.subplot(121)
plt.imshow(img, cmap='Greys')

plt.subplot(122)
plt.imshow(np.squeeze(res), cmap='Greys')

# Or we could specify the exact dimensions we want to visualize:
# plt.imshow(res[0, :, :, 0], cmap='Greys')

## Exercise 1

1. define an arbitrary square filter (could be random or could be of your choice)
- convolve it with the image as displayed above
- display the image, the filter and the convolved image

In [ ]:
random_filter = tf.Variable(tf.random_normal([4, 4, 1, 1]))
convolved_2 = tf.nn.conv2d(img_4d, random_filter,
                           strides=[1, 1, 1, 1],
                           padding='SAME')
convolved_2

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    res = convolved_2.eval()
    random_filter_res = random_filter.eval()
print(res.shape)

In [ ]:
plt.subplot(131)
plt.imshow(img, cmap='Greys')

plt.subplot(132)
plt.imshow(random_filter_res.squeeze(), cmap='Greys')

plt.subplot(133)
plt.imshow(np.squeeze(res), cmap='Greys')

## A simple convolutional model

In [ ]:
tf.reset_default_graph()

In [ ]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

## Exercise 2

create some wrappers for simplicity

1. create a function `conv2d(x, W, b, strides=1)` that creates a convolutional layer using the `tf.nn.conv2d` and `tf.nn.bias_add`.
- create a function `maxpool2d(x, k=2)` that performs max pooling using the `tf.nn.max_pool` function, where k is the stride in both directions.


In [ ]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W,
                     strides=[1, strides, strides, 1],
                     padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x,
                          ksize=[1, k, k, 1],
                          strides=[1, k, k, 1],
                          padding='SAME')

Create the model

In [ ]:
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    with tf.name_scope('conv_net'):
        # Convolution Layer
        conv1 = conv2d(x, weights['wc1'], biases['bc1'])
        # Max Pooling (down-sampling)
        conv1 = maxpool2d(conv1, k=2)

        # Convolution Layer
        conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
        # Max Pooling (down-sampling)
        conv2 = maxpool2d(conv2, k=2)

        # Fully connected layer
        # Reshape conv2 output to fit fully connected layer input
        fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
        fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
        fc1 = tf.nn.relu(fc1)
        # Apply Dropout
        fc1 = tf.nn.dropout(fc1, dropout)

        # Output, class prediction
        out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

Store layers weight & biases

In [ ]:
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32]), name='wc1'),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64]), name='wc2'),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024]), name='wd1'),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, 10]), name='wout')
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32]), name='bc1'),
    'bc2': tf.Variable(tf.random_normal([64]), name='bc2'),
    'bd1': tf.Variable(tf.random_normal([1024]), name='bd1'),
    'out': tf.Variable(tf.random_normal([10]), name='bout')
}

Construct model, define loss, optimizer, evaluation, initializer etc..

In [ ]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

### Launch the graph

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))